## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Vaini,TO,-21.2000,-175.2000,80.76,89,40,12.66,scattered clouds
1,1,Karratha,AU,-20.7377,116.8463,84.00,29,3,15.77,clear sky
2,2,Naze,JP,28.3667,129.4833,79.72,68,100,15.50,overcast clouds
3,3,Castro,BR,-24.7911,-50.0119,64.06,96,100,0.54,overcast clouds
4,4,Hualmay,PE,-11.0964,-77.6139,63.07,86,50,9.60,scattered clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is your minimum temperature preference?"))

max_temp = float(input("What is your maximum temperature preference?"))

What is your minimum temperature preference?50
What is your maximum temperature preference?85


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                      (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Vaini,TO,-21.2000,-175.2000,80.76,89,40,12.66,scattered clouds
1,1,Karratha,AU,-20.7377,116.8463,84.00,29,3,15.77,clear sky
2,2,Naze,JP,28.3667,129.4833,79.72,68,100,15.50,overcast clouds
3,3,Castro,BR,-24.7911,-50.0119,64.06,96,100,0.54,overcast clouds
4,4,Hualmay,PE,-11.0964,-77.6139,63.07,86,50,9.60,scattered clouds
5,5,Saint George,US,37.1041,-113.5841,79.63,35,1,1.01,clear sky
6,6,Butaritari,KI,3.0707,172.7902,81.63,69,7,14.18,clear sky
7,7,Pisco,PE,-13.7000,-76.2167,62.65,82,0,13.80,clear sky
8,8,Kutum,SD,14.2000,24.6667,73.36,23,100,7.25,overcast clouds
9,9,Mahebourg,MU,-20.4081,57.7000,70.25,88,40,8.05,scattered clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                459
City                   459
Country                454
Lat                    459
Lng                    459
Max Temp               459
Humidity               459
Cloudiness             459
Wind Speed             459
Current Description    459
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

clean_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Vaini,TO,-21.2000,-175.2000,80.76,89,40,12.66,scattered clouds
1,1,Karratha,AU,-20.7377,116.8463,84.00,29,3,15.77,clear sky
2,2,Naze,JP,28.3667,129.4833,79.72,68,100,15.50,overcast clouds
3,3,Castro,BR,-24.7911,-50.0119,64.06,96,100,0.54,overcast clouds
4,4,Hualmay,PE,-11.0964,-77.6139,63.07,86,50,9.60,scattered clouds
...,...,...,...,...,...,...,...,...,...,...
663,663,Moerai,PF,-22.4333,-151.3333,75.76,72,100,6.91,overcast clouds
665,665,El Campo,US,29.1966,-96.2697,80.28,56,2,6.69,clear sky
666,666,Nouadhibou,MR,20.9310,-17.0347,75.18,69,0,19.57,clear sky
667,667,Jinchang,CN,38.4953,102.1739,51.73,62,11,2.10,few clouds


In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Vaini,TO,80.76,scattered clouds,-21.2000,-175.2000,
1,Karratha,AU,84.00,clear sky,-20.7377,116.8463,
2,Naze,JP,79.72,overcast clouds,28.3667,129.4833,
3,Castro,BR,64.06,overcast clouds,-24.7911,-50.0119,
4,Hualmay,PE,63.07,scattered clouds,-11.0964,-77.6139,
5,Saint George,US,79.63,clear sky,37.1041,-113.5841,
6,Butaritari,KI,81.63,clear sky,3.0707,172.7902,
7,Pisco,PE,62.65,clear sky,-13.7000,-76.2167,
8,Kutum,SD,73.36,overcast clouds,14.2000,24.6667,
9,Mahebourg,MU,70.25,scattered clouds,-20.4081,57.7000,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found. Skipping...")

Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...


In [9]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df[hotel_df["Hotel Name"] != '']
clean_hotel_df.head(20)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Vaini,TO,80.76,scattered clouds,-21.2000,-175.2000,Keleti Beach Resort
1,Karratha,AU,84.00,clear sky,-20.7377,116.8463,ibis Styles Karratha
2,Naze,JP,79.72,overcast clouds,28.3667,129.4833,オーベルジュ・アガルイ
3,Castro,BR,64.06,overcast clouds,-24.7911,-50.0119,CHACARA BAILLY
4,Hualmay,PE,63.07,scattered clouds,-11.0964,-77.6139,Hotel Villa Kitzia Huacho
5,Saint George,US,79.63,clear sky,37.1041,-113.5841,Best Western Plus Abbey Inn
6,Butaritari,KI,81.63,clear sky,3.0707,172.7902,Isles Sunset Lodge
7,Pisco,PE,62.65,clear sky,-13.7000,-76.2167,Hotel San Isidro Oficial
8,Kutum,SD,73.36,overcast clouds,14.2000,24.6667,Dibbo areaمنطقة دبو
9,Mahebourg,MU,70.25,scattered clouds,-20.4081,57.7000,Shandrani Beachcomber Resort & Spa


In [10]:
# 8a. Create the output File (CSV)
output_data_file = "Vacation_Search/WeatherPy_Vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [12]:
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
# 11b. Display the figure
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.4)
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(height='420px'))